***
# Post-Block Assignment 1
***
Johan Hanekom <br>
20780893 <br>
16/11/2021
***
## Setup

In [1]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

run('pip install --upgrade pip')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')


>> pip install --upgrade pip

>> pip install --quiet apache-beam

>> mkdir -p data



# Import libraries

In [2]:
import apache_beam as beam
import re
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
import pandas as pd

from apache_beam import Create, FlatMap, Map, ParDo, Flatten, Partition
from apache_beam import Values, CoGroupByKey
from apache_beam import pvalue, window, WindowInto
from datetime import datetime

# Read in the data

In [3]:
! curl -o data/users_v.csv https://storage.googleapis.com/bdt-beam/users_v.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0  7794k      0 --:--:-- --:--:-- --:--:-- 7794k


In [4]:
! curl -o data/orders_v.csv https://storage.googleapis.com/bdt-beam/orders_v.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36.0M  100 36.0M    0     0  36.0M      0  0:00:01  0:00:01 --:--:-- 36.0M


In [5]:
# What columns are in users?
! cat data/users_v.csv | head

user_id,name,gender,age,address,date_joined
1,Anthony Wolf,male,73,New Rachelburgh-VA-49583,2019/03/13
2,James Armstrong,male,56,North Jillianfort-UT-86454,2020/11/06
3,Cody Shaw,male,75,North Anne-SC-53799,2004/05/29
4,Sierra Hamilton,female,76,New Angelafurt-ME-46190,2005/08/26
5,Chase Davis,male,31,South Bethmouth-WI-18562,2018/04/30
6,Sierra Andrews,female,21,Ryanville-MI-69690,2007/05/25
7,Ann Stone,female,41,Smithmouth-SD-17340,2005/01/05
8,Karen Santos,female,34,Mariaville-AK-29888,2003/12/12
9,Ronald Meyer,male,41,North Cherylhaven-NJ-04197,2015/11/14


In [6]:
# What columns are in orders?
! cat data/orders_v.csv | head

order_no,user_id,product_list,date_purchased
1000,1887,Cassava,2000-01-01
1001,838,"Calabash, Water Spinach",2000-01-01
1002,2032,"Onion, Rapini",2000-01-01
1003,1482,"Swiss Chard, Artichoke",2000-01-01
1004,475,"Turnip Greens, Plantain",2000-01-01
1005,1627,"English Cucumber, Parsley Root, Cauliflower",2000-01-01
1006,2000,"Bell Pepper, English Cucumber",2000-01-01
1007,2099,Arugula,2000-01-01
1008,2337,"Shallots, Jerusalem Artichoke",2000-01-01


In [7]:
# Create smaller users data
! head -100 data/users_v.csv > data/small_users.csv

# Create smalledr orders data
! head -100 data/orders_v.csv > data/small_orders.csv

# Question 4
## Question 4.1
Merge the 2 input files. [3]
### Define the functions

In [8]:
class format_users(beam.DoFn):
    def process(self, element):
        user_id, name, gender, age, address, date_joined= element.split(",")
        return [{
            "user_id": str(user_id),
            "name": str(name),
            "gender": str(gender),
            "age": str(age),
            "address": str(address.replace("-", ";")),
            "date_joined": str(date_joined.replace("/", "-"))
        }]
     
class format_orders(beam.DoFn):
    def process(self, element):
        words = element.split(",")
        order_id = words[0]
        user_id = words[1]
        date_purchased = words[-1]
        product_array = words[2:-1]

        sent_str = ""
        for i in product_array:
            sent_str += str(i) + ";"
        sent_str = sent_str[:-1]

        product_list = sent_str
        return [{
            "user_id": str(user_id),
            "order_id": str(order_id),
            "product_list": str(product_list),
            "date_purchased": str(date_purchased.replace("/", "-"))
        }]


class ToCsv(beam.DoFn):

    def process(self, element):
      yield ','.join(dict(element).values())


### Define Pipeline

In [9]:
with beam.Pipeline() as pipeline:
  orders_collection = (pipeline | "Read Orders" >> beam.io.ReadFromText('data/orders_v.csv', skip_header_lines=0)
                        | "Format Orders" >> ParDo(format_orders())
                        | "to_csv orders" >> beam.ParDo(ToCsv())
                        | "Save orders" >> beam.io.WriteToText("data/orders_cleaned.csv")
                        )
  
with beam.Pipeline() as pipeline:
  users_collection = (pipeline | "Read Users" >> beam.io.ReadFromText('data/users_v.csv', skip_header_lines=0)
                        | "Format Users" >> ParDo(format_users())
                        | "to_csv users" >> beam.ParDo(ToCsv())
                        | "Save users" >> beam.io.WriteToText("data/users_cleaned.csv")
                        )



In [10]:
! ls data

merged.csv			   small_orders.csv
orders_cleaned.csv-00000-of-00001  small_users.csv
orders_v.csv			   users_cleaned.csv-00000-of-00001
small_data.csv			   users_v.csv


### Join via pandas

I really can't figure out how to join the two...

In [11]:
df_users = pd.read_csv("data/users_cleaned.csv-00000-of-00001", header = 0,names = ['user_id', 'name', 'gender', 'age', 'address', 'date_joined'])
df_orders = pd.read_csv("data/orders_cleaned.csv-00000-of-00001", header = 0,names = ['user_id','order_no','product_list','date_purchased'])

In [12]:
df_orders.head(1)

,user_id,order_no,product_list,date_purchased
0,1887,1000,Cassava,2000-01-01


In [13]:
df_users.head(1)

,user_id,name,gender,age,address,date_joined
0,1,Anthony Wolf,male,73,New Rachelburgh;VA;49583,2019-03-13


In [14]:
len(df_orders)

806193

In [15]:
len(df_users)

2357

In [16]:
# df_mergded = pd.merge(df_users,df_orders,on='user_id')
df_merge= pd.merge(df_orders,df_users,on='user_id')
df_merge.head(3)

,user_id,order_no,product_list,date_purchased,name,gender,age,address,date_joined
0,1887,1000,Cassava,2000-01-01,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1,1887,1356,Bitter Melon; Turnip; Water Chestnut,2000-01-04,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
2,1887,2151,English Cucumber; Plantain; Taro,2000-01-12,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06


In [17]:
df_merge["product_list"]

0                                      Cassava
1         Bitter Melon; Turnip; Water Chestnut
2             English Cucumber; Plantain; Taro
3                        Bell Pepper; Shallots
4                   Watercress; Hearts of Palm
                          ...                 
806188                    Grape Leaves; Tomato
806189        Peppers; Hearts of Palm; Parsnip
806190                        Avocado; Spinach
806191                       Lettuce; Purslane
806192                            Okra; Garlic
Name: product_list, Length: 806193, dtype: object

In [18]:
# Save
df_merge.to_csv("data/merged.csv", index = False)

## Question 4.2
Perform a transformation that determines the average number of orders for female and male customers,
respectively.

### Create a sample for testing

In [19]:
# Create smaller users data
! head -1000 data/merged.csv > data/small_data.csv
! cat data/small_data.csv | head

user_id,order_no,product_list,date_purchased,name,gender,age,address,date_joined
1887,1000,Cassava,2000-01-01,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,1356,Bitter Melon; Turnip; Water Chestnut,2000-01-04,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,2151,English Cucumber; Plantain; Taro,2000-01-12,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,3075,Bell Pepper; Shallots,2000-01-20,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,5406,Watercress; Hearts of Palm,2000-02-11,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,7815,Swiss Chard; Fiddlehead; Fennel,2000-03-11,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,11147,Tomato; Mustard Greens; Lotus Seed; Grape Leaves,2000-04-12,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,15456,Fennel,2000-05-26,Andrea Hartman,female,58,Karenmouth;MN;95314,2016-07-06
1887,16842,Celtuce; Cabbage,2000-06-03,Andrea Hartman,female,58,Karenmouth;MN;95314,20

## Define pipelines

In [20]:
header = ['user_id','order_no','product_list','date_purchased','name','gender','age','address','date_joined']

class Transform(beam.DoFn):

  def process(self, element):
    # data = dict(zip(header, element.split(',')))
    row_list =  element.split(',')
    yield [row_list[0],row_list[1],row_list[5]]


with beam.Pipeline() as pipeline:
  (pipeline 
      | 'Read lines' >> beam.io.ReadFromText('data/merged.csv', skip_header_lines=1)
      | 'format' >> beam.ParDo(Transform())
      | beam.GroupBy(lambda s: s[0])
      | 'add_key' >> beam.Map(lambda elem: (elem[1][0][2], len(elem[1])) )
      | 'sum' >> beam.combiners.Mean.PerKey()
      | beam.Map(print)
  )

('female', 342.1982608695652)
('male', 341.89312344656173)


## Question 4.3
Perform a transformation that groups users into age groups [16-26), [26-36), [36-46), [46-56), and determine the total number of orders placed by customers in each age group.

In [21]:
header = ['user_id','order_no','product_list','date_purchased','name','gender','age','address','date_joined']

class Transform(beam.DoFn):

  def process(self, element):
    # data = dict(zip(header, element.split(',')))
    row_list =  element.split(',')

    age = int(row_list[6])
    age_group = "other"

    if (age >= 16) and (age<26):
      age_group = "Ages [16-26)"

    if (age >= 26) and (age<36):
      age_group = "Ages [26-36)"

    if (age >= 36) and (age<46):
      age_group = "Ages [36-46)"

    if (age >= 46) and (age<56):
      age_group = "Ages [46-56)"


    yield [row_list[0],row_list[1],age_group]


with beam.Pipeline() as pipeline:
  (pipeline 
      | 'Read lines' >> beam.io.ReadFromText('data/merged.csv', skip_header_lines=1)
      | 'format' >> beam.ParDo(Transform())
      | 'drop-header' >> beam.Filter(lambda x: x[2] != 'other')
      | beam.GroupBy(lambda s: s[0])
      | 'add_key' >> beam.Map(lambda elem: (elem[1][0][2], len(elem[1])) )
      | 'sum' >> beam.CombinePerKey(sum)
      | beam.Map(print)
  )

('Ages [26-36)', 137232)
('Ages [46-56)', 117701)
('Ages [36-46)', 132048)
('Ages [16-26)', 96019)


## Question 4.4

In [22]:
'ExtractWords' >> beam.FlatMap(lambda x: re.findall(r"\bmale\b|\bfemale\b", x))

header = ['user_id','order_no','product_list','date_purchased','name','gender','age','address','date_joined']

class Transform(beam.DoFn):

  def process(self, element):
    # data = dict(zip(header, element.split(',')))
    row_list =  element.split(',')

    age = int(row_list[6])
    products = row_list[2]
    age_group = "other"
    is_spinach = "No"

    if "Spinach" in products:
      is_spinach = "Yes"

    if (age >= 16) and (age<26):
      age_group = "Ages [16-26)"

    if (age >= 26) and (age<36):
      age_group = "Ages [26-36)"

    if (age >= 36) and (age<46):
      age_group = "Ages [36-46)"

    if (age >= 46) and (age<56):
      age_group = "Ages [46-56)"


    yield [row_list[0],row_list[1],age_group, is_spinach]


with beam.Pipeline() as pipeline:
  (pipeline 
      | 'Read lines' >> beam.io.ReadFromText('data/merged.csv', skip_header_lines=1)
      | 'format' >> beam.ParDo(Transform())
      | 'drop-age' >> beam.Filter(lambda x: x[2] != 'other')
      | 'drop-not spinach' >> beam.Filter(lambda x: x[3] != 'No')
      | beam.GroupBy(lambda s: s[0])
      | 'add_key' >> beam.Map(lambda elem: (elem[1][0][2], len(elem[1])) )
      | 'sum' >> beam.CombinePerKey(sum)
      | beam.Map(print)
  )

('Ages [26-36)', 8786)
('Ages [46-56)', 7584)
('Ages [36-46)', 8415)
('Ages [16-26)', 6110)
